In [ ]:
import os
from database_tools.tools import BuildDatabase

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

config = dict(
    low=0.5,
    high=8.0,
    sim1=0.8,
    sim2=0.9,
    snr_t=22.5,
    hr_diff=1/6,
    f0_low=0.667,
    f0_high=3.0,
)

worker = BuildDatabase(
    output_dir='data-2022-09-25/',
    config=config,
    win_len=1024,
    fs=125,
    samples_per_file=1,
    max_samples=1,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)

df = worker.run()

In [ ]:
import os
import pandas as pd
from database_tools.tools import DataEvaluator

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

df = pd.read_csv('data-2022-09-23/mimic3_stats.csv')
worker = DataEvaluator(stats=df)

figs = worker.run()

In [ ]:
import glob
import json
import numpy as np

ppg = []
abp = []

with open('../data-2022-09-25/mimic3/mimic3_0000008.jsonlines', 'r') as f:
    for sample in f:
        sample = json.loads(sample)
        ppg.append(sample['ppg'])
        abp.append(sample['abp'])


ppg = np.array(ppg).flatten()
abp = np.array(abp).flatten()

In [ ]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

fig = go.FigureWidget()
fig.add_scatter()
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Amplitude',
    width=1000,
    height=800,
)
fig

In [ ]:
i = 2004
y = abp[i * 1024: (i + 1) * 1024 - 524]
fig.data[0].y = y

In [ ]:
# for f in $files; do mongoimport --db MIMICDB --file $f; done

from pymongo import MongoClient

uri = 'mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.6.0'
client = MongoClient(uri)

coll = client.MIMICDB.mimic3_0000067

cursor = coll.find()
for doc in cursor:
    print(doc)